In [1]:
import sys
import os
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Cm
from docx.shared import RGBColor
from docx.shared import Pt

# Labels used for evaluation
LABELS = {'Згоден': 5, 'Скоріше згоден': 4, 'Важко відповісти': 3, 'Скоріше не згоден': 2, 'Не згоден': 1}
# Categories used for evaluation of a subject
CATEGORIES_SUBJECT = (
            'Наповнення навчальної дисципліни є сучасним та актуальним',
            'Навчальна дисципліна забезпечує підготовку до майбутньої професійної діяльності',
            'Навчально-методичне забезпечення дисципліни є достатнім'
        )
#Categories used for evaluation of a tecaher
CATEGORIES_TEACHER = (
            'Є компетентним',
            'Подає навчальний матеріал логічно і доступно',
            'Оцінює об\'єктивно',
            'Відкритий до запитань та дискусій, підтримує контакт з аудиторією',
            'Враховує потреби та майбутній фах студентів',
            'Є тактовним, забезпечує комфортний мікроклімат'
        )
# Programs and their shortened titles in excel
PROGRAMS = {
    'Біоінформатика': 'Біоінформатика і структурна біологія, Магістр',
    'Біологія ВТ': 'Біологія (високі технології), Бакалавр',
    'Біотех': 'Високі технології (Біотехнологія), Магістр',
    'ВТ ПФ': 'Високі технології (Прикладна фізика та наноматеріали), Магістр',
    'Хемо': 'Високі технології (Хемоінформатика), Магістр',
    'ВТ Хімія': 'Високі технології (Хімія та наноматеріали), Магістр',
    'Електроніка': 'Електроніка (високі технології), Бакалавр',
    'Нанофізика': 'Нанофізика та комп\'ютерні технології, Бакалавр',
    'Хімія ВТ': 'Хімія (високі технології), Бакалавр'
}

# During the analysis we will find subjects to put them into the dict via their names and groups
# Key is the name in the following format: 'subject program_shortened semester' = 'Mathematics Biology HT 3'
# Each key corresponds to the object of the Subject class
subjects = {}
# During the analysis we will find teachers and subjects they teach (a list) and save them in a dict
# Key is the string with the name of the teacher
# Each key corresponds to the list of subjects in the following format: 'subject program_shortened semester'
teachers = {}
# Similarly for programs we will save their names and objects
# Key is the string with full name of the program
# # Each key corresponds to the object of the Program class
programs = {}

# List of departments and teachers
DEPARTMENTS = {
    'ТОВТ': [
        'Лозовський В.З.',
        'Колежук О.К.',
        'Шило С.О.',
        'Разумова М.А.',
        'Васильєв Т.А.',
        'Вишивана І.Г.',
        'Мішакова Т.О.',
        'Завертаний Т.Л.',
        'Чегель В.І.',
        'Гринько Д.О.',
        'Васильєв А.Г.'
    ],
    'НФКС': [
        'Скришевський В.А.',
        'Шкавро А.Г.',
        'Ільченко В.В.',
        'Євтух А.А.',
        'Гаврильченко І.В.',
        'Іванов І.І.',
        'Рєзніков М.І.',
        'Сусь Б.Б.',
        'Вербицький В.Г.',
        'Пилипова О.В.',
        'Лисоченко С.В.',
        'Русінчук Н.М.'
    ],
    'МББІ': [
        'Нипорко О.Ю.',
        'Цимбалюк О.В.',
        'Давидовська Т.Л.',
        'Данилович Ю.В.',
        'Драган А.І.',
        'Самофалова Д.О.',
        'Футорна О.А.',
        'Войтешенко І.С.',
        'Дзюбенко Н.В.',
        'Світін Р.С.',
        'Компанець Т.А.',
        'Іванова В.Д.',
        'Солдаткін О.П.',
        'Дзядевич С.В.',
        'Пірко Н.М.',
        'Горобчишин В.А.',
        'Джаган В.В.',
        'Петльована В.Р.',
        'Рибалка І.Є.',
        'Сухопара С.В.'
    ],
    'СХ': [
        'Рябухін С.В.',
        'Волочнюк Д.М.',
        'Комаров І.В.',
        'Толстанова Г.М.',
        'Маханькова В.Г.',
        'Гринь С.В.',
        'Булавко Г.В.',
        'Грабчук Г.П.',
        'Цуварєв О.Ю.',
        'Ляпунов О.Ю.',
        'Чередніченко А.',
        'Цюпа К.С.',
        'Іщенко О.О.',
        'Шиванюк О.М.',
        'Роженко О.Б.'
    ]
}

In [2]:
# Classes

# Functions used in classes
# Labeling the bars
def autolabel(rects, b):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        b.annotate('{}'.format(height),
                    xy = (rect.get_x() + rect.get_width() / 2, height),
                    xytext = (0, 2),  # 3 points vertical offset
                    textcoords = "offset points",
                    ha = 'center', va = 'bottom',
                    fontsize = 20)

# Adding line breaks to titles
def process_title(string):
    a = string.split()
    if len(a) > 5:
        string = ' '.join([elem for elem in a[:5]])
        string += '\n'
        string += ' '.join([elem for elem in a[5:]])
    return string

# Class for subjects
class Subject:
    # Is defined by the title of the subject, the group and semester of teaching, and takes the results of the poll
    def __init__(self, title, group, semester, data):
        self.title = title
        # Group in excel data is mentioned shortened, we need to put the whole one from the dict
        self.group = PROGRAMS[group]
        self.semester = semester
        self.data = data
        # Number of people interviewed is needed for further publishing
        self.interviewed = len(self.data)
        # Average mark for each point
        self.means = np.round(self.data.mean(axis=0), 2)
        # Marks regarding the subject itself
        self.rates = pd.Series(np.asarray(self.means[:3]), index = CATEGORIES_SUBJECT)
        # Average mark
        self.total = np.round(np.average(self.rates), 2)
        # The list of responds (element == element is checking for NaN objects)           
        self.responds = [element for element in self.data.iloc[:, -1] if element == element]
        
        # Constructing the list of  teachers
        def find_teachers(self):
            t = {}
            n = int((len(self.means) - 3)/6)
            for i in range(n):
                k = 3 + 6*i
                name = self.means.index[k]
                name = name[9:name.find('[')-1]
                t[name] = Teacher_rate(name, 
                                   self.title, 
                                   self.group, 
                                   self.semester,
                                   self.interviewed,
                                   np.round(self.means[k:k + 6], 2), 
                                   self.responds)
            return t
        # and put them as an object property
        self.teachers = find_teachers(self)
        
        # Method for ploting the results via the subject
    def plot_results(self):        
        fig, [a, b] = plt.subplots(nrows=1, ncols=2, figsize=(18,9))
        plt.subplots_adjust(top = 0.8)
        subj_title = process_title(self.title)
        fig.suptitle(f'{self.semester} семестр\n{subj_title}', 
                     y = 1, 
                     verticalalignment='top',
                     weight = 'bold',
                     fontsize = 28, 
                     linespacing = 1.5)
        Categories = ['1', '2', '3']
        rects1 = a.bar(Categories, self.rates, color='b', align='center', edgecolor='black', width=0.5)
        a.set_title('Результати опитування', size = 20)
        a.set_ylabel('Середня оцінка',size=22)
        a.set_xlabel('Категорія', size=22)
        a.grid(True)
        a.set_ylim(0, 5.5)
        a.set_xticklabels(Categories, size=20)
        a.set_yticklabels([0, 1, 2, 3, 4, 5], size=20)
        autolabel(rects1, a)
        b.text(0.5, 0.85, 'Середня оцінка:', 
               fontsize = 30, 
               horizontalalignment = 'center', 
               verticalalignment='bottom')
        b.text(0.5, 0.55, self.total, 
               fontsize = 90,
               color = 'red',
               weight = 'bold',
               horizontalalignment = 'center', 
               verticalalignment='bottom')   
        b.text(0.5, 0.45, f'Опитано студентів: {self.interviewed}', 
               fontsize = 30,
               horizontalalignment = 'center', 
               verticalalignment='top')
        b.text(0.05, 0.3,
              'Категорії:\n'
            + '1. Наповнення навчальної дисципліни є сучасним\n    та актуальним\n'
            + '2. Навчальна дисципліна забезпечує підготовку до\n    майбутньої професійної діяльності\n'
            + '3. Навчально-методичне забезпечення дисципліни\n    є достатнім\n',
               fontsize = 14,
               horizontalalignment = 'left', 
               verticalalignment='top')
        b.set_xticks([])
        b.set_yticks([])
        figadress = f'{self.title}, програма {self.group}, {self.semester}.png'
        fig.savefig(figadress)
        plt.close()
        return figadress
    
    # Method for adding information to the document
    def add_to_doc(self, document):
        # Creating figure with the results
        pict = self.plot_results()
        # Add new paragraph with central alignment
        p = document.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = p.add_run()
        # Ass picture
        run.add_picture(pict, width=Cm(15))
        # Remove picture from the disk
        os.remove(pict)
        # Add information for teachers
        for teacher in self.teachers.keys():
            self.teachers[teacher].add_to_doc(document, for_prog = True)
        # Add responds
        document.add_paragraph('Відгуки студентів:')
        for resp in self.responds:
            document.add_paragraph(resp)
        

# Class for rates of the teacher        
class Teacher_rate:
    # It is defined by the name of the teacher, 
    # the subject, group and semester of teaching, 
    # number of people interviewed 
    # and the list of the responds
    def __init__ (self, name, subject, group, semester, interviewed, rates, responds):
        self.name = name
        self.subject = subject
        # Average mark for each point
        self.rates = pd.Series(np.asarray(rates), index = CATEGORIES_TEACHER)
        self.responds = responds
        self.group = group
        self.semester = semester
        self.interviewed = interviewed
        # Average mark
        self.total = np.round(np.average(rates), 2)
    
    # Method for ploting the results via the subject
    def plot_results(self, for_prog = False):        
        fig, [a, b] = plt.subplots(nrows=1, ncols=2, figsize=(18,9))
        plt.subplots_adjust(top = 0.75)
        subj_title = process_title(self.subject)
        suptitle = f'{self.name}' if for_prog else f'{self.group}\n{subj_title}\n{self.semester} семестр'
        fig.suptitle(suptitle, 
                     y = 1, 
                     verticalalignment='top',
                     weight = 'bold',
                     fontsize = 28, 
                     linespacing = 1.5)
        Categories = ['1', '2', '3', '4', '5', '6']
        rects1 = a.bar(Categories, self.rates, color='b', align='center', edgecolor='black', width=0.5)
        a.set_title('Результати опитування', size = 20)
        a.set_ylabel('Середня оцінка',size=22)
        a.set_xlabel('Категорія', size=22)
        a.grid(True)
        a.set_ylim(0, 5.5)
        a.set_xticklabels(Categories, size=20)
        a.set_yticklabels([0, 1, 2, 3, 4, 5], size=20)
        autolabel(rects1, a)
        b.text(0.5, 0.85, 'Середня оцінка:', 
               fontsize = 30, 
               horizontalalignment = 'center', 
               verticalalignment='bottom')
        b.text(0.5, 0.55, self.total, 
               fontsize = 90,
               color = 'red',
               weight = 'bold',
               horizontalalignment = 'center', 
               verticalalignment='bottom')   
        b.text(0.5, 0.45, f'Опитано студентів: {self.interviewed}', 
               fontsize = 30,
               horizontalalignment = 'center', 
               verticalalignment='top')
        b.text(0.04, 0.35, 
              'Категорії:\n'
            + '1. Є компетентним\n'
            + '2. Подає навчальний матеріал логічно і доступно\n'
            + '3. Оцінює об\'єктивно\n'
            + '4. Відкритий до запитань та дискусій,\n   підтримує контакт з аудиторією\n'
            + '5. Враховує потреби та майбутній фах студентів\n'
            + '6. Є тактовним, забезпечує комфортний мікроклімат', 
               fontsize = 14,
               horizontalalignment = 'left', 
               verticalalignment='top')
        b.set_xticks([])
        b.set_yticks([])
        figadress = f'{self.name}, предмет {self.subject}.png'
        fig.savefig(figadress)
        plt.close()
        return figadress
    
    # Method for adding information to the document
    def add_to_doc(self, document, for_prog = False):
        # Creating figure with the results
        pict = self.plot_results(for_prog)
        # Add new paragraph with central alignment
        p = document.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = p.add_run()
        # Ass picture
        run.add_picture(pict, width=Cm(15))
        # Remove picture from the disk
        os.remove(pict)
        # Add responds if needed
        if not for_prog:
            document.add_paragraph('Відгуки студентів:')
            for resp in self.responds:
                document.add_paragraph(resp)
        
# Class for programs
class Program:
    # It is defined by the name of the program, and the list of subjects via semesters
    # Creating new program we put the lists empty
    def __init__(self, prog_title):
        self.prog_title = prog_title
        self.subj_list = {}
        
    # Define the method for adding the subject to the semester
    def add_subj(self, subj_title, semester):
        if semester not in self.subj_list.keys():
            self.subj_list[semester] = []
        self.subj_list[semester].append(subj_title)
        
    # Method for adding information to the document
    def add_to_doc(self, document):
        # Define amount of semesters
        n_sem = len(self.subj_list)
        # Add info for each semester
        for i in range(1, n_sem + 1):
            add_title(f'{i} семестр', document)
            for subject in self.subj_list[str(i)]:
                subjects[subject].add_to_doc(document)
            document.add_page_break()

In [3]:
# Functions

# Function for creating the list of subjects and the list of teachers
def find_teach_and_subjects(file_name):
    
    # Import the file
    xl = pd.ExcelFile(file_name)
    # Take the list of sheet names
    sheets = xl.sheet_names

    # Loop over all sheets in the file
    for sheet in sheets:
        # Taking the data
        data = xl.parse(sheet)

        # Changing label by the corresponding points mentioned in LABELS
        for key in LABELS.keys():
            data.replace(key, LABELS[key], inplace = True)

        # i is the number of the column analyzed
        i = 0
        # start consists of the column number, where the data for the current subject starts
        # when starting analysis we set it to 0
        start = 0

        # As we will pass some columns, it is not neseccary to use 'for i in range(n)' loop
        while True:
            # Taking the name of the column
            name = data.columns[i]

            # If this is the first column for some subject
            if name[:6] == 'Якість' and name[-3:] == 'им]':
                # And if this is not the first subject in list
                if start != 0:
                    # Add new object Subject using previously defined title in "subject"
                    # The sheet name consits of the information on the group and semester of teaching
                    subjects[f'{subject} {sheet}'] = Subject(subject, sheet[:-2], sheet[-1:], data.iloc[:, start:i])
                    # Hence, we can to change start and subject values for the new one 
                # Set new starting column for the next subject
                start = i
                # Clear all unneseccary info from the column title
                name = name.replace('Якість освітньої компоненти ', '')
                name = name.replace(' [Наповнення навчальної дисципліни є сучасним та актуальним]', '')
                # And set new subject name
                subject = name
                # Next two columns are not neseccary for us now, so go further on three steps
                i += 3

            # Else if this is the last column
            elif i == len(data.columns) - 1:
                    # Create new subject based on previously defined title and starting point
                    subjects[f'{subject} {sheet}'] = Subject(subject, sheet[:-2], sheet[-1:], data.iloc[:, start:])
                    # Break the loop
                    break

            # Else if this column is the starting one for the teacher
            elif name[:8] == 'Викладач':
                # Define its name
                name = name[9:name.find('[')-1]
                # If this is new teacher which is not in our dict yet
                if name not in teachers:
                    # Add it to the dict with an empty list
                    teachers[name] = []
                # For teacher in a dict add subject and group to its list of subjects
                teachers[name].append(f'{subject} {sheet}')
                # Next 5 are not neseccary for us now, so go further on six steps
                i += 6

            # In another case just go further on one step
            else:
                i += 1
                
                
# Function for completing the list of programs with semesters and subjects
def complete_programs():
    # Create objects for programs
    for key in PROGRAMS.keys():
        prog_title = PROGRAMS[key]
        programs[prog_title] = Program(prog_title)
    # Loop over all the subjects
    for key in subjects.keys():
        # Add subject name into the proper semester of the program
        prog_title = subjects[key].group
        subj_title = key
        semester = subjects[key].semester
        programs[prog_title].add_subj(subj_title, semester)
        

# Function for addind the title to the document
def add_title(title, document):
    p = document.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = p.add_run(f'{title}\n\n')
    run.font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
    run.font.size = Pt(30)
    run.bold = True
        
        
# Function for converting file from docx to pdf
def convert_to_pdf(file_name):
    wdFormatPDF = 17
    in_file = os.path.abspath(f'{file_name}.docx')
    out_file = os.path.abspath(f'{file_name}.pdf')
    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(in_file)
    doc.SaveAs(out_file, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()
    os.remove(f'{file_name}.docx')
    
    
# Function for creating a report on the departments
def make_report_deps():
    for dept in DEPARTMENTS.keys():
        # Create new document and put the title
        document = Document()
        # Need to include each teacher from the department
        for teacher in DEPARTMENTS[dept]:
            # The title is the teacher's name
            add_title(teacher, document)
            p = document.add_paragraph()
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            # Include results separately for each subject
            for subject in teachers[teacher]:
                # Take Subject object via key
                # Then take an attribute 'teacher' consisting the dict of teachers
                # Take Teacher object via key
                # Use method 'add_to_doc' for including inforamtion to the document
                subjects[subject].teachers[teacher].add_to_doc(document)
            # Page break before new teacher
            document.add_page_break()
        document.save(f'Відгуки по кафедрі {dept}.docx')
        convert_to_pdf(f'Відгуки по кафедрі {dept}')    
        
# Function for creating a report on the programs
def make_report_progs():
    for prog in programs.keys():
        # Create new document and put the title
        document = Document()
        # Add info to document
        programs[prog].add_to_doc(document)
        document.save(f'Відгуки по ОП {prog}.docx')
        convert_to_pdf(f'Відгуки по ОП {prog}')
        
# Final function
def process_poll_results(file_name):
    find_teach_and_subjects(file_name)
    complete_programs()
    make_report_deps()
    make_report_progs()

In [4]:
# Put in the file name
file_name = 'Опитування по предметам і викладачам.xlsx'
process_poll_results(file_name)

<ipython-input-2-b927c155897a>:172: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_xticklabels(Categories, size=20)
<ipython-input-2-b927c155897a>:173: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_yticklabels([0, 1, 2, 3, 4, 5], size=20)
<ipython-input-2-b927c155897a>:82: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_xticklabels(Categories, size=20)
<ipython-input-2-b927c155897a>:83: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_yticklabels([0, 1, 2, 3, 4, 5], size=20)
